In [1]:
# Add / Run this cell first to fix the issue with environment variables for Java not being set properly
# Even though they're in .bash_profile
# TODO: Fix this

%run '../../spark_variables.ipynb'

In [2]:
#Initialize and load weather dataframe

from pyspark import SparkContext
sc = SparkContext(master="local[4]")
#sc.version

import os
import sys

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType
%pylab inline

# Just like using Spark requires having a SparkContext, using SQL requires an SQLContext
sqlContext = SQLContext(sc)

Populating the interactive namespace from numpy and matplotlib


In [3]:
from os.path import split,join,exists
from os import mkdir,getcwd,remove
from glob import glob

# create directory if needed


data_dir='../../Data'
weather_dir=join(data_dir,'Weather')
!ls $weather_dir

NY.parquet NY.tgz


In [4]:
if exists(weather_dir):
    print('directory',weather_dir,'already exists')
else:
    print('making',weather_dir)
    mkdir(weather_dir)

directory ../../Data/Weather already exists


In [5]:
file_index='NY'
zip_file='%s.tgz'%(file_index) #the .csv extension is a mistake, this is a pickle file, not a csv file.
old_files='%s/%s*'%(weather_dir,zip_file[:-3])
for f in glob(old_files):
    print('removing',f)
    !rm -rf {f}

command="curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state/%s > %s/%s "%(zip_file, weather_dir,zip_file)
print(command)
!$command
!ls -lh $weather_dir/$zip_file

removing ../../Data/Weather/NY.tgz
removing ../../Data/Weather/NY.parquet
curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state/NY.tgz > ../../Data/Weather/NY.tgz 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63.2M  100 63.2M    0     0  31.1M      0  0:00:02  0:00:02 --:--:-- 31.1M
-rw-r--r--  1 brianroepke  staff    63M Sep 16 06:34 ../../Data/Weather/NY.tgz


In [6]:
#extracting the parquet file
!tar zxvf {weather_dir}/{zip_file} -C {weather_dir}

x NY.parquet/
x NY.parquet/_SUCCESS
x NY.parquet/part-00022-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
x NY.parquet/part-00000-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
x NY.parquet/part-00021-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
x NY.parquet/part-00001-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
x NY.parquet/part-00023-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
x NY.parquet/part-00002-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
x NY.parquet/part-00024-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
x NY.parquet/part-00003-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
x NY.parquet/part-00025-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
x NY.parquet/part-00004-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
x NY.parquet/part-00027-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
x NY.parquet/part-00005-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.

In [7]:
weather_parquet = join(weather_dir,zip_file[:-3]+'parquet')
print(weather_parquet)
df = sqlContext.read.load(weather_parquet)
df.show(1)

../../Data/Weather/NY.parquet
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|    Station|Measurement|Year|              Values|       dist_coast|      latitude|         longitude|        elevation|state|             name|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|USW00094704|   PRCP_s20|1945|[00 00 00 00 00 0...|361.8320007324219|42.57080078125|-77.71330261230469|208.8000030517578|   NY|DANSVILLE MUNI AP|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
only showing top 1 row



## Comparing GroupBy and ReduceByKey

In [8]:
def Use_Iterator(X):
    key,iter=X
    s=0
    for pair in iter:
        s+= pair[1]
    return (key,s)

In [9]:
%%time
A=df.select('Station','Year').rdd.map(lambda row: (row['Station'],row['Year']))

CPU times: user 2.13 ms, sys: 1.35 ms, total: 3.48 ms
Wall time: 102 ms


In [10]:
A.take(3)

[('USW00094704', 1945), ('USW00094704', 1946), ('USW00094704', 1947)]

In [11]:
AA=A
for i in range(100):
    AA=AA.union(A)
print(A.count(),AA.count())

168398 17008198


In [12]:
AA.getNumPartitions(),A.getNumPartitions()

(404, 4)

In [13]:
AA=AA.repartition(10).cache()

In [14]:
AA.getNumPartitions(),A.getNumPartitions()

(10, 4)

In [15]:
B=AA.groupBy(lambda pair:pair[0]).map(Use_Iterator)
B.take(3)

[('USC00305334', 150190434),
 ('USC00309100', 71564560),
 ('USC00303319', 251199120)]

In [16]:
C=B.collect()
C[:4]

[('USC00305334', 150190434),
 ('USC00309100', 71564560),
 ('USC00303319', 251199120),
 ('USC00306538', 178714652)]

In [17]:
%%time
g=AA.reduceByKey(lambda x,y:x+y)

CPU times: user 4.98 ms, sys: 3.14 ms, total: 8.12 ms
Wall time: 36.8 ms


In [18]:
%%time
C2=g.collect()

CPU times: user 4.53 ms, sys: 1.76 ms, total: 6.29 ms
Wall time: 3.34 s


In [19]:
C2[:4]

[('USC00305334', 150190434),
 ('USC00309100', 71564560),
 ('USC00303319', 251199120),
 ('USC00306538', 178714652)]

In [20]:
!pip install pyarrow

     |████████████████████████████████| 34.4MB 1.3MB/s eta 0:00:01


In [21]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [22]:
@pandas_udf("Station string, Measurement string, Year int, State string", PandasUDFType.GROUPED_MAP)
def substract_mean(pdf):
    # pdf is a pandas.DataFrame
    return len(pdf.columns)

In [ ]:
df.drop('Values').groupby("Station").apply(substract_mean).show(1)

In [24]:
df.schema

StructType(List(StructField(Station,StringType,true),StructField(Measurement,StringType,true),StructField(Year,LongType,true),StructField(Values,BinaryType,true),StructField(dist_coast,DoubleType,true),StructField(latitude,DoubleType,true),StructField(longitude,DoubleType,true),StructField(elevation,DoubleType,true),StructField(state,StringType,true),StructField(name,StringType,true)))

In [25]:
df.show(1)

+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|    Station|Measurement|Year|              Values|       dist_coast|      latitude|         longitude|        elevation|state|             name|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
|USW00094704|   PRCP_s20|1945|[00 00 00 00 00 0...|361.8320007324219|42.57080078125|-77.71330261230469|208.8000030517578|   NY|DANSVILLE MUNI AP|
+-----------+-----------+----+--------------------+-----------------+--------------+------------------+-----------------+-----+-----------------+
only showing top 1 row



In [26]:
"Station string, Measurement string, Year int, Values binary, State string"

'Station string, Measurement string, Year int, Values binary, State string'